<a href="https://colab.research.google.com/github/flindwall/home_sales_pyspark/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import os
# # Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.3.1'
# spark_version = 'spark-3.3.1'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# # Start a SparkSession
# import findspark
# findspark.init()

In [2]:
!apt-get install openjdk-8-jdk-headless -qq
!pip install pyspark==3.2.0

from pyspark.sql import SparkSession

Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Setting up libxtst6:amd64 (2:1.2.3-1) ...
Setting up openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header = True, inferSchema = True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_sales')


In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select date_built, round(avg(price), 2) as average_price
from home_sales
where bedrooms = 4
group by date_built
order by date_built
""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select date_built, round(avg(price), 2) as average_price
from home_sales
where bedrooms = 3 and bathrooms = 3
group by date_built
order by date_built
""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select date_built, round(avg(price), 2) as average_price
from home_sales
where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
group by date_built
order by date_built
""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

spark.sql("""
select round(avg(view), 2) as avg_view_rating
from home_sales
where price >= 350000
""").show()


start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.0001697540283203125 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select round(avg(view), 2) as avg_view_rating
from home_sales
where price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 1.3684468269348145 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_df.write.parquet('date_built', mode = 'overwrite')

In [18]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('date_built')

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_date')

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
select round(avg(view), 2) as avg_view_rating
from parquet_data
where price >=350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.6365573406219482 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales").unpersist()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False